<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivosII/blob/master/notebooks/4e_learningToRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Algoritmos learning to rank

El objetivo de esta libreta es estudiar los algoritmos de learning to rank en sistemas de recomendación

[Créditos](https://github.com/liyinxiao/rankerNN2pmml)


## RankNet

In [ ]:
import keras
from keras import backend as K
from keras.layers import Activation, Dense, Input, Subtract
from keras.models import Model
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""Generación de datos dummies"""
INPUT_DIM = 3

#se definen los objetos a comparar
X1 = 2 * np.random.uniform(size=(50, INPUT_DIM)) #Objeto 1
X2 = np.random.uniform(size=(50, INPUT_DIM)) #Objeto 2
Y = [random.randint(0,1) for _ in range(50)]

print("Tamaño de X1",X1.shape)
print("Tamaño de X2",X2.shape)
print("Tamaño de Y",len(Y))

Tamaño de X1 (50, 3)
Tamaño de X2 (50, 3)
Tamaño de Y 50


In [ ]:
# Transformación de los datos
mms = MinMaxScaler() #default=(0, 1)
mms.fit(np.concatenate((X1, X2), axis=0))
X1 = mms.transform(X1)
X2 = mms.transform(X2)
Y = np.asarray(Y)

In [ ]:
#Visualizando la entrada
df_input = pd.DataFrame(X1, columns = ['Feature1', 'Feature2', 'Feature3'])
df_input

,Feature1,Feature2,Feature3
0,0.235954,0.171641,0.146817
1,0.748380,0.363056,0.815422
2,0.305907,0.450836,0.968545
3,0.848680,0.900038,0.839035
4,0.113283,0.354020,0.843931
5,0.217854,0.290997,0.295289
6,0.856920,0.660012,0.081989
7,0.647708,0.105392,0.284839
8,0.032633,0.484461,0.123438
9,0.591712,0.012010,1.000000


In [ ]:
def RankNet_model(input_shape):
    """Construcción de la red neuronal"""
    h1 = Dense(4, activation="relu", name='Relu_layer1') #red completamente conectada
    h2 = Dense(2, activation='relu', name='Relu_layer2')
    h3 = Dense(1, activation='linear', name='Identity_layer')
    # Objeto 1 a comparar
    input1 = Input(shape=(input_shape,), name='Input_layer1')
    x1 = h1(input1)
    x1 = h2(x1)
    x1 = h3(x1)
    # Objeto 2 a comparar
    input2 = Input(shape=(input_shape,), name='Input_layer2')
    x2 = h1(input2)
    x2 = h2(x2)
    x2 = h3(x2)
    # Capa de comparación
    subtracted = Subtract(name='Subtract_layer')([x1, x2])
    # Función de activación
    out = Activation('sigmoid', name='Activation_layer')(subtracted)
    # Modelo
    model = Model(inputs=[input1, input2], outputs=out)
    return model

In [ ]:
model = RankNet_model(INPUT_DIM)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_layer1 (InputLayer)       [(None, 3)]          0                                            
__________________________________________________________________________________________________
Input_layer2 (InputLayer)       [(None, 3)]          0                                            
__________________________________________________________________________________________________
Relu_layer1 (Dense)             (None, 4)            16          Input_layer1[0][0]               
                                                                 Input_layer2[0][0]               
__________________________________________________________________________________________________
Relu_layer2 (Dense)             (None, 2)            10          Relu_layer1[0][0]            

In [ ]:
# Entrenamiento del modelo
model.compile(optimizer="adam", loss="binary_crossentropy")
model.fit([X1,X2], Y, batch_size=10, epochs=5, verbose=1)

Epoch 1/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6959
Epoch 2/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6952
Epoch 3/5
5/5 [==============================] - 0s 2ms/step - loss: 0.6950
Epoch 4/5
5/5 [==============================] - 0s 3ms/step - loss: 0.6944
Epoch 5/5
5/5 [==============================] - 0s 2ms/step - loss: 0.6942


In [ ]:
#Salida del modelo
get_ranker_output = K.function([model.layers[0].input], [model.layers[-3].get_output_at(0)])
Ranker_output = get_ranker_output([X1])[0]

In [ ]:
#Visualizando la salida
df_output = pd.DataFrame(Ranker_output, columns = ['score'])
df_output

,score
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.021078
9,0.000000


## LambdaRank

In [ ]:
!pip install LambdaRankNN
from LambdaRankNN import LambdaRankNN

#Generación de datos dummies
X = np.array([[0.2, 0.3, 0.4],
              [0.1, 0.7, 0.4],
              [0.3, 0.4, 0.1],
              [0.8, 0.4, 0.3],
              [0.9, 0.35, 0.25]])
y = np.array([0, 1, 0, 0, 2])
qid = np.array([1, 1, 1, 2, 2])

#Entrenamiento del modelo
ranker = LambdaRankNN(input_size=X.shape[1], hidden_layer_sizes=(16,8,), 
                      activation=('relu', 'relu',), 
                      solver='adam')
ranker.fit(X, y, qid, epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 456ms/step - loss: 0.2085
Epoch 2/5
1/1 [==============================] - 0s 15ms/step - loss: 0.2083
Epoch 3/5
1/1 [==============================] - 0s 5ms/step - loss: 0.2080
Epoch 4/5
1/1 [==============================] - 0s 4ms/step - loss: 0.2077
Epoch 5/5
1/1 [==============================] - 0s 4ms/step - loss: 0.2075
ndcg: 0.75


In [ ]:
y_pred = ranker.predict(X)
ranker.evaluate(X, y, qid, eval_at=2)

ndcg@2: 0.5
